# **PFIRS Enrichment Processing**

## **Install Required Python Libraries**

You are editing this notebook, so we assume you have already cloned the repository from  
[https://github.com/klinucsd/interagency-tracking-system](https://github.com/klinucsd/interagency-tracking-system) into the folder ~/work/interagency-tracking-system. Please verify that the `requirements.txt` file is available in that folder before proceeding.

In [1]:
# ! pip install -r requirements.txt

## **Download PFIRS data in zip format**

In [ ]:
import sys
import os
# Add parent directory to Python path to import project modules
sys.path.insert(0, os.path.abspath('..'))

from utils.download_from_box import download_box_shared_link

pfirs_box_link = "https://calfire.box.com/s/jpe3gz0y2y7kub7cq7lmv5mga9yc03su"
download_box_shared_link(pfirs_box_link)

## **Unzip PFIRS data in GDB format**

In [ ]:
! [ -d PFIRS2024.gdb ] || unzip PFIRS2024.gdb.zip

## **Check the PFIRS GDB to Find Its Layers**

In [5]:
! ogrinfo -so PFIRS2024.gdb

INFO: Open of `PFIRS2024.gdb'
      using driver `OpenFileGDB' successful.
Layer: PFIRS2024 (Point)


## **Download Lookup Table**

In [12]:
lookup_box_link = "https://calfire.box.com/s/opn58bkf1iirnj25pducpx6zdf6xucjs"
download_box_shared_link(lookup_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip
    ✓ Saved: PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip (1,867,971 bytes)



['./PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip']

## **Unzip Reference Data in GDB format**

In [ ]:
! [ -d a_Reference.gdb ] || unzip ../../a_Reference.gdb.zip

## **Check the Reference GDB to Find Its Layers**

In [6]:
! ogrinfo -so a_Reference.gdb

INFO: Open of `a_Reference.gdb'
      using driver `OpenFileGDB' successful.
Layer: Crosswalk (None)
Group a_Reference:
  Layer: WFR_TF_Template (Multi Polygon)
  Layer: California (Multi Polygon)
  Layer: Counties (Multi Polygon)
  Layer: Broad_Vegetation_Types (Multi Polygon)
  Layer: CALFIRE_Ownership_Update (Multi Polygon)
  Layer: WUI (Multi Polygon)
  Layer: WFRTF_Regions (Multi Polygon)
  Layer: Own_Veg_Region_WUI (Multi Polygon)
Group b_Originals:
Group c_Enriched:
Group d_Appended:
Group e_Transformed:
Group f_Report_Data:


## **Enriching PFIRS Data**

Converts PFIRS actual treatments dataset into the Task Force standardized schema. Dataset is enriched with vegetation, ownership, county, WUI, Task Force Region, and year. 

In [ ]:
import warnings
import logging
import time
from datetime import datetime
import psutil
import os
import sys

# Add parent directory to Python path to import project modules
sys.path.insert(0, os.path.abspath('..'))

from enrich.enrich_PFIRS import enrich_PFIRS

# Get the current process ID
process = psutil.Process(os.getpid())

pfirs_input_gdb_path = "PFIRS2024.gdb"
pfirs_input_layer_name = "PFIRS2024"
treat_poly_gdf_path = ""
treat_poly_layer_name = ""
a_reference_gdb_path = "a_Reference.gdb"
start_year, end_year = 2010, 2025
output_gdb_path = f"/tmp/PFIRS_{start_year}_{end_year}.gdb"
output_layer_name = f"PFIRS_enriched_{datetime.today().strftime('%Y%m%d')}"

enrich_PFIRS(pfirs_input_gdb_path,
             pfirs_input_layer_name,
             None,
             a_reference_gdb_path,
             start_year,
             end_year,
             output_gdb_path,
             output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
print(f"Memory usage: {memory_usage:.2f} MB")
print(f"PFIRS enrichment GDB: {output_gdb_path}")
print(f"PFIRS enrichment Layer: {output_layer_name}")